In [1]:
import numpy as np
from scipy.integrate import RK45,solve_ivp
from ODE_potentials import VanDerPolePotential,LotkiVolterraPotential
from ODE_samplers import MALA_ODE,ULA_ODE,grad_ascent_ODE,run_eval_test,set_function
from multiprocessing import Pool
import multiprocessing
from zv_cv import Eval_ZVCV

from baselines import construct_ESVM_kernel
from optimize import optimize_parallel_new
from utils import *

Parameters for van-der-Pole and Lotki-Volterra examples: 

In [2]:
typ = 'LV' #'LV' for Lotki-Volterra, 'VdP' for Van-der-Pole
method = {"sampler":"MALA"} #switch between ULA and MALA
f_type = "posterior_mean"

if typ == 'VdP':
    #true parameter value
    theta_star = 1.0
    #initial coordiante and speed
    y0 = np.array([0.0,2.0],dtype=float)
    #error of measurements
    sigma = 0.5
    #prior variance
    sigma_prior = 0.5
elif typ == 'LV':
    theta_star = np.array([0.6,0.025,0.8,0.025],dtype = float)
    #initial number of victims and predators
    y0 = np.array([30.0,4.0],dtype=float)
    #setting prior parameters
    sigma_prior = np.array([0.5,0.05,0.5,0.05],dtype = float)
    mu_prior = np.array([1.0,0.05,1.0,0.05],dtype=float)
    #measurements error
    sigma = np.array([0.25,0.25])

Timestaps

In [3]:
#initial and last time moments
t0 = 0
t_bound = 10
#moments of observations
t_moments = np.arange(t_bound+1)

Creating potentials

In [4]:
if typ == 'VdP':
    Cur_pot = VanDerPolePotential(sigma,sigma_prior,t_moments,theta_star,y0,t0,t_bound)
elif typ == 'LV':
    Cur_pot = LotkiVolterraPotential(sigma,mu_prior,sigma_prior,t_moments,theta_star,y0,t0,t_bound)

system solved
[[ 37.52358801   5.44451059]
 [ 49.31015054   4.44056815]
 [ 68.02780239   7.49382876]
 [ 96.31994597  69.0224216 ]
 [ 33.96485552 119.57915384]
 [ 11.19476367  55.29111344]
 [  7.28510513  25.46629586]
 [  3.95843583  11.12542133]
 [ 12.06748946   3.71380176]
 [ 12.38341447   6.01556616]
 [ 18.28774199   3.71351383]]


Sampling (currently with MALA)

In [5]:
r_seed = 666
#burn-in period
N_burn = 1*10**3
#Train size
N_train = 1*10**4
#Test size
N_test = 1*10**4
#number of test trajectories
n_traj_test = 100
if typ == 'VdP':
    #dimension
    d = 1
    #step size
    step = 1e-3
elif typ == 'LV':
    #dimension
    d = 4
    #step size
    step = 5e-6

### Construct kernels and sample

In [6]:
# Construct kernel
#W_train_spec = construct_ESVM_kernel(N_train)
#W_test_spec = construct_ESVM_kernel(N_test)

In [7]:
if typ == 'VdP':
    params_prior = {"sigma":sigma_prior}
elif typ == 'LV':
    params_prior = {"mu":mu_prior,"sigma":sigma_prior}

### Compute starting point (maximum likelihood)

In [ ]:
N_steps_ascent = 5000
traj,traj_grad = grad_ascent_ODE(1453,Cur_pot,step,params_prior,N_steps_ascent,d,typ)
theta_mle = traj[-1,:]
print("mle for parameters: ",theta_mle)
Cur_pot.set_mle(theta_mle)

In [ ]:
print(traj_grad[-10:,:])

### Setting function 

In [ ]:
if f_type == "posterior_mean":
    inds_arr = np.array([0,1,2,3])
    params = None 
    t_moments = None
    if method["sampler"] == "ULA":
        traj,traj_grad = ULA_ODE(r_seed,Cur_pot, step, params_prior, N_burn, N_train, d, typ)
    elif method["sampler"] == "MALA":
        traj,traj_grad,n_accepted = MALA_ODE(r_seed,Cur_pot, step, params_prior, N_burn, N_train, d, typ)
        print("n_accepted = ",n_accepted)
    traj = traj[np.newaxis,:]
    traj_grad = traj_grad[np.newaxis,:]
    f_vals = set_function(f_type,traj,inds_arr,params)
    print(f_vals.shape)
elif f_type == "evidence":
    #time moments for numeric integrations
    n_points = 10.0
    f_vals = np.zeros((1+int(n_points),N_train,1),dtype = float)
    traj = np.zeros((1+int(n_points),N_train,d),dtype = float)
    traj_grad = np.zeros((1+int(n_points),N_train,d),dtype = float)
    t_moments = (np.arange(n_points+1)/n_points)**5
    print(t_moments)
    for i in range(len(t_moments)):
        if method["sampler"] == "ULA":
            f_vals[i],traj[i],traj_grad[i] = ULA_ODE(r_seed+i,Cur_pot, step, params_prior, N_burn, N_train, d, typ,t_moments[i])
        elif method["sampler"] == "MALA":
            f_vals[i],traj[i],traj_grad[i],n_accepted = MALA_ODE(r_seed+i,Cur_pot,step,params_prior,N_burn,N_train,d,typ,t_moments[i])
            print("n_accepted = ",n_accepted)           

In [ ]:
inds_arr = np.array([0])
params = None
t_moments = None
r_seed = 777
n_traj = 24
traj = []
traj_grad = []
#genertae data
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(MALA_ODE, [(r_seed+i,Cur_pot, step, params_prior, N_burn, N_train, d, typ) for i in range (n_traj)])
trav.close()
for i in range(len(res)):
    traj.append(res[i][0])
    traj_grad.append(res[i][1])
    print("accepted = ",res[i][2])
traj = np.asarray(traj)
traj_grad = np.asarray(traj_grad)
traj_grad = (-1)*traj_grad

In [ ]:
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Eval_ZVCV, [(traj[i,:,:],traj_grad[i,:,:],f_type) for i in range (n_traj)])
trav.close()

In [ ]:
res_arr = np.asarray(res)
print(res_arr.shape)

### Plotting the results

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith ZV-1', 'MALA \nwith CV-1']

In [ ]:
data = [res_arr[:,0],res_arr[:,1],res_arr[:,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['MALA \nwith ZV-1', 'MALA \nwith CV-1']

In [ ]:
data = [res_arr[:,1],res_arr[:,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith ZV-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr[:,0],res_arr[:,2],res_arr[:,4]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['MALA \nwith ZV-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr[:,2],res_arr[:,4]] 
boxplot_ind(data, title, labels)